**1. Reading Data**

In [ ]:
import pandas as pd
import numpy as np
X=pd.read_csv('/kaggle/input/covid19/COVID20DATA (1).csv')
#X.describe()
#X.head()

**2. Name of the states and corresponding populations**

In [ ]:
#The all states
all_states=['Andaman and Nicobar Islands','Andhra Pradesh','Arunachal Pradesh','Assam','Bihar','Chandigarh','Chhattisgarh',
            'Delhi','Goa','Gujarat','Haryana','Himachal Pradesh','Jammu and Kashmir','Jharkhand','Karnataka','Kerala','Ladakh',
            'Madhya Pradesh', 'Maharashtra','Manipur','Meghalaya','Mizoram','Odisha','Puducherry','Punjab','Rajasthan',
            'Tamil Nadu','Telengana','Tripura','Uttarakhand','Uttar Pradesh','West Bengal']
total_populations=[417036,53903393,1570458,35607039,124799926,1158473,29436231,
           18710922,1586250,63872399,28204692,7451955,13606320,38593948,67562686,35699443,289023,
           85358965,123144223, 3091545,3366710,1239244,46356334,1413542,30141373,81032689,
           77841267,38510982,4169794,11250858,237882725,99609303]#ref:https://uidai.gov.in/images/state-wise-aadhaar-saturation.pdf
migran=[3829, 32571.0, 2871.0, 426441.0, 1500612.0, 12554, 362553,157879, 3520, 703641, 1289.0, 1880, 48780.0,
          530047.0, 523526, 311124.0, 50, 7535811.0, 1829910.0, 12338.0, 4266.0,
          15750.0, 579351, 174791, 51564.0, 1308130.0, 72145.0, 37050.0, 34247.0,
          100862, 3249638.0, 1384693.0 ]
number_of_states=len(all_states)
populations=[0 for i in range(number_of_states)]
for i in range(number_of_states):
    populations[i]=total_populations[i]-migran[i]
print(populations)    
#print("The total number of states we are going to consider is {}.".format(number_of_states))


**3. Simulating migration data using two different Gaussian distribution**


In [ ]:
import math
mu1=15
mu2=40
def m(M):
    sum=0
    for day in range(50):
        a=-((day-mu1)**2)/1800
        sum+=math.exp(1.2*a)
        b=-((day-mu2)**2)/1800
        sum+=math.exp(0.8*b)
    for day in range(50,70):
        a=-((day-mu1)**2)/1800
        sum+=math.exp(0.8*a)
        b=-((day-mu2)**2)/1800
        sum+=math.exp(1.2*b)
    for day in range(30,75):
        b=-((day-mu2)**2)/1800
        sum+=math.exp(b)
        
    k=(M*60*((2*math.pi)**0.5)) /sum   
    return k
def mig(M,day):
    s=0
    k=m(M)
    if day in range(50):
        a=math.exp(-((day-mu1)**2)/1800)
        b=math.exp(-((day-mu2)**2)/1800)
        c=((0.8*a+1.2*b)*k)/(60*(2*math.pi)**0.5)
        return c 
    if day in range(50,75):
        a=math.exp(-((day-mu1)**2)/1800)
        b=math.exp(-((day-mu2)**2)/1800)
        c=((1.2*a+0.8*b)*k)/(60*(2*math.pi)**0.5)
        return c
    if day in range(30,75):
        b=math.exp(-((day-mu2)**2)/1800)
        c=((b)*k)/(60*(2*math.pi)**0.5)    
        return c  
def total_mig(M,day):
    a=int(mig(M,0))
    for i in range(day):
        a+=int(mig(M,day))
    return a    

**4. Separating statewise data and calculating migrated individuals, current confirmed indiviuals, current recovered and susceptibles**

In [ ]:
state_wise_data=[0 for i in range(number_of_states)]
for i in range(number_of_states):
    state_wise_data[i]=X.loc[X['STATE'] == all_states[i]]


def add_col(D,j):
    n=D.shape[0]
    migration=np.array([int(mig(migran[j],i)) for i in range(n)])
    D.insert(5,"MIGRATION",migration)
    
    total_migration=np.array([total_mig(migran[j],i) for i in range(n)])
    D.insert(5, "TOTAL MIGRATION",total_migration)
    
    population=np.array([populations[j] for i in range(n)])
    D.insert(6,'POPULATION',population)
    
    D.insert(7,'CURRENT POPULATION',D["POPULATION"]+D["MIGRATION"])

    D.insert(8,'RECOVERED',D["DEATH"]+D["CURE"])
    
    D.insert(9,'CURRENT CONFIRMED',D["CONFIRMED"]-D["RECOVERED"])
    
    D.insert(10,'SUSCEPTABLE',D["CURRENT POPULATION"]-D["CONFIRMED"])
    

for i in range(number_of_states):
    add_col(state_wise_data[i],i)
    state_wise_data[i].to_csv(all_states[i])
       
state_wise_data[31]

**4. Generating output files which is not necessary**

In [ ]:
out_files=['./'+all_states[i] for i in range(number_of_states)]
#print(out_files)

def sus(D):
    n=D.shape[0]
    ##########################
    alpha=[0 for i in range(n)]
    a=0
    alpha[0]=a
    for i in range(1,n):
        b=D["SUSCEPTABLE"][i]*D['CURRENT CONFIRMED'][i]
        if b!=0:
            alpha[i]=(D["SUSCEPTABLE"][i-1]-D["SUSCEPTABLE"][i])*D["POPULATION"][0]/b
        else:
            alpha[i]==0
    D.insert(8,'ALPHA',alpha)
    ###########################
    beta=[0 for i in range(n)]
    a=0
    beta[0]=a
    for i in range(1,n):
        if (D["DEATH"][i]+D['CURE'][i])!=0:
            beta[i]=1-(D['CURE'][i-1]+D["DEATH"][i-1])/(D["DEATH"][i]+D['CURE'][i])
            
        else:
            beta[i]==0
    D.insert(9,'BETA',beta)

**5. Visualizing data**

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

def visual(D,i):
    
    fig = px.line(D, x = 'DATE', y = ['RECOVERED','CURRENT CONFIRMED','MIGRATION']#,'SUSCEPTABLE','CURRENT POPULATION']
                  , title='Day Wise Cases of {}'.format(all_states[i]))
    return fig.show()
def cons(D,i):
    fig = px.line(D, x = 'DATE', y = ['ALPHA','BETA']#,'SUSCEPTABLE','CURRENT POPULATION']
                  , title='Constants of {}'.format(all_states[i]))
    return fig.show()
    
def f(D,i):
    fig = px.line(D, x = 'DATE', y = ['SUSCEPTABLE']#,'CURRENT POPULATION']
                  , title='Susceptibles of {}'.format(all_states[i]))
    return fig.show()
A=[]
B=[]
for i in range(32):
    D=pd.read_csv(out_files[i])
    sus(D)
    visual(D,i)
    #f(D,i)
    #cons(D,i)
    print("Average value of alpha=",np.mean(D.ALPHA))
    print("Average value of beta=",np.mean(D.BETA))
    A.append(np.mean(D.ALPHA))
    B.append(np.mean(D.BETA))
    #print("{} & {} & {}\\".format(all_states[i],round(np.mean(D.ALPHA),6),round(np.mean(D.BETA),6)))
#print('Average alpha for India:',np.mean(A))  
#print('Average beta for India:',np.mean(B))